In [1]:
import numpy as np
%cd  '/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0'
%pwd

camelyon_train_comp = np.load('camelyon_train_ds.npz')

/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0


In [2]:
camelyon_train = camelyon_train_comp['arr_0']

np.random.shuffle(camelyon_train)

In [7]:
# np.random.shuffle(camelyon_train)
# camelyon_train_5  = camelyon_train[:int(len(camelyon_train)/20)]
# np.random.shuffle(camelyon_train)
# camelyon_train_10 = camelyon_train[:int(len(camelyon_train)/10)]
np.random.shuffle(camelyon_train)
camelyon_train_15 = camelyon_train[:int(len(camelyon_train)/(20/3))]
np.random.shuffle(camelyon_train)
camelyon_train_20 = camelyon_train[:int(len(camelyon_train)/5)]
np.random.shuffle(camelyon_train)
camelyon_train_25 = camelyon_train[:int(len(camelyon_train)/4)]
np.random.shuffle(camelyon_train)
camelyon_train_30 = camelyon_train[:int(len(camelyon_train)/(10/3))]
np.random.shuffle(camelyon_train)
camelyon_train_35 = camelyon_train[:int(len(camelyon_train)/(20/7))]
np.random.shuffle(camelyon_train)
camelyon_train_40 = camelyon_train[:int(len(camelyon_train)/(5/2))]
np.random.shuffle(camelyon_train)
camelyon_train_45 = camelyon_train[:int(len(camelyon_train)/(20/9))]
np.random.shuffle(camelyon_train)
camelyon_train_50 = camelyon_train[:int(len(camelyon_train)/2)]
np.random.shuffle(camelyon_train)
camelyon_train_55 = camelyon_train[:int(len(camelyon_train)/(20/11))]
np.random.shuffle(camelyon_train)
camelyon_train_60 = camelyon_train[:int(len(camelyon_train)*0.6)]
np.random.shuffle(camelyon_train)
camelyon_train_65 = camelyon_train[:int(len(camelyon_train)*0.65)]
np.random.shuffle(camelyon_train)
camelyon_train_70 = camelyon_train[:int(len(camelyon_train)*0.7)]
np.random.shuffle(camelyon_train)
camelyon_train_75 = camelyon_train[:int(len(camelyon_train)*0.75)]
np.random.shuffle(camelyon_train)
camelyon_train_80 = camelyon_train[:int(len(camelyon_train)*0.8)]
np.random.shuffle(camelyon_train)
camelyon_train_85 = camelyon_train[:int(len(camelyon_train)*0.85)]
np.random.shuffle(camelyon_train)
camelyon_train_90 = camelyon_train[:int(len(camelyon_train)*0.9)]
np.random.shuffle(camelyon_train)
camelyon_train_95 = camelyon_train[:int(len(camelyon_train)*0.95)]
np.random.shuffle(camelyon_train)

In [ ]:
camelyon_train_45.shape

In [4]:
import json 
with open('/home/ubuntu/image-drift-monitoring/config/common/drift_detection_config.json') as config_file:
        drift_detection_config = json.load(config_file)

In [5]:
%cd  '/home/ubuntu/image-drift-monitoring'
%pwd
from src.modules.alibi_detect.untrained_encoder import UntrainedAutoencoder

/home/ubuntu/image-drift-monitoring


2023-02-10 16:27:57.690248: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 16:27:57.690310: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ubuntu/miniconda3/envs/alibitorchwhy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_5.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_5,detector_name='Camelyon_UAE_5_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_10.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_10,detector_name='Camelyon_UAE_10_MMD',save_dec=True)

myUAE = UntrainedAutoencoder(drift_detection_config)
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_15.shape[1:])
myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_15,detector_name='Camelyon_UAE_15_MMD',save_dec=True)

myUAE = UntrainedAutoencoder(drift_detection_config)
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_20.shape[1:])
myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_20,detector_name='Camelyon_UAE_20_MMD',save_dec=True)

myUAE = UntrainedAutoencoder(drift_detection_config)
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_25.shape[1:])
myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_25,detector_name='Camelyon_UAE_25_MMD',save_dec=True)

myUAE = UntrainedAutoencoder(drift_detection_config)
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_30.shape[1:])
myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_30,detector_name='Camelyon_UAE_30_MMD',save_dec=True)

myUAE = UntrainedAutoencoder(drift_detection_config)
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_35.shape[1:])
myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_35,detector_name='Camelyon_UAE_35_MMD',save_dec=True)

myUAE = UntrainedAutoencoder(drift_detection_config)
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_40.shape[1:])
myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_40,detector_name='Camelyon_UAE_40_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_45.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_45,detector_name='Camelyon_UAE_45_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_50.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_50,detector_name='Camelyon_UAE_50_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_55.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_55,detector_name='Camelyon_UAE_55_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_60.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_60,detector_name='Camelyon_UAE_60_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_65.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_65,detector_name='Camelyon_UAE_65_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_70.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_70,detector_name='Camelyon_UAE_70_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_75.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_75,detector_name='Camelyon_UAE_75_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_80.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_80,detector_name='Camelyon_UAE_80_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_85.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_85,detector_name='Camelyon_UAE_85_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_90.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_90,detector_name='Camelyon_UAE_90_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_95.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train_95,detector_name='Camelyon_UAE_95_MMD',save_dec=True)

# myUAE = UntrainedAutoencoder(drift_detection_config)
# myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train.shape[1:])
# myUAE.init_detector(detector_type='MMD',reference_data=camelyon_train,detector_name='Camelyon_UAE_100_MMD',save_dec=True)


# H0 Hypothesis testing

In [ ]:
myUAE.make_prediction(target_data=camelyon_train_15,detector_type='CVM')

# drift testing

In [ ]:
camelyon_test_comp = np.load('/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0/camelyon_test_ds.npz')
camelyon_test = camelyon_test_comp['arr_0']

np.random.shuffle(camelyon_test)

In [ ]:
myUAE.make_prediction(target_data=camelyon_test,detector_type='KS')